In [1]:
!pip install tetgen
import igl
import numpy as np
import meshplot as mp
import scipy.sparse as sp
import scipy
import scipy.spatial
import tetgen

In [2]:
v, f = igl.read_triangle_mesh("data/vol.obj")
bbv, bbf = igl.read_triangle_mesh("data/src.obj")
bbVNew, bbFNew = igl.read_triangle_mesh("data/trg.obj")

#bbvN = igl.per_vertex_normals(bbv, bbf)
#bbv += 5 * bbvN

#bbvN = igl.per_vertex_normals(bbv, bbf)
#bbv += 5 * bbvN

p = mp.plot(v, f, shading={"wireframe": True})
p.add_edges(bbv, bbf, shading={"line_color": "red"});
#p.add_points(bbv, shading={"point_color": "red"})
p.add_edges(bbVNew, bbFNew, shading={"line_color": "green"});
#p.add_points(bbVNew, shading={"point_color": "green"})

p = mp.plot(bbv, bbf, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(4.8249053…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.7979431…

In [3]:
newVs = np.zeros_like(v)

gi = np.zeros((bbv.shape[0], v.shape[0]))

spaceV = np.concatenate((v, bbv), axis = 0)
spaceF = bbf + len(v)
tetGen = tetgen.TetGen(spaceV, spaceF)
nodes, elems = tetGen.tetrahedralize(quality = False)
b = igl.boundary_facets(elems)
p = mp.plot(nodes, elems, shading={"wireframe": True})

laplace = igl.cotmatrix(nodes, elems)
for x in range(len(v), len(v) + len(bbv)):
    laplace[x, :] = 0
    laplace[x, x] = 1

#solve laplace for each cage vertex giving the weights for each point w.r.t the cage vertex
for x in range(len(bbv)):
    # to solve for a cade vertex
    soln = np.zeros(spaceV.shape[0])
    soln[len(v) + x] = 1
    wi = sp.linalg.spsolve(laplace, soln)
    gi[x] = wi[:len(v)]

newVs = np.zeros_like(v)
#compute each new point p'
for x in range(len(v)):
    #iterate over each gi(p) for each Ci'
    for y in range(len(bbv)):
        newVs[x] += gi[y][x] * bbVNew[y]

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.7979431…

D:\anaconda\lib\site-packages\scipy\sparse\_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [4]:
p = mp.plot(newVs, f, shading={"wireframe": True})
p.add_edges(bbv, bbf, shading={"line_color": "red"});
#p.add_points(bbv, shading={"point_color": "red"})
p.add_edges(bbVNew, bbFNew, shading={"line_color": "green"});
#p.add_points(bbVNew, shading={"point_color": "green"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-11.19636…